In [1]:
import sys, os, time, getpass

sys.path.append("/home/tatiane/lib/")

import pessoal
from pessoal import *
print('AppID: ', sc.applicationId)

# Pedir ao spark para exibir só erros, escondendo avisos (WARN) e infos
# O correto é resolver a causa como, por exemplo: ordenados = Window.partitionBy().orderBy(dfs[i].columns[0])
spark.sparkContext.setLogLevel("ERROR")

Tempo inicial da execucao: 2025-07-02 16:41:08.492903
User: tatiane
Node: Inspiron-3583
AppID:  local-1751485213258


# Censo escolar 2023

In [2]:
# Leitura com encoding apropriado para o conteúdo da base
censo = spark.read.csv('/home/tatiane/Documentos/censo_escolar2/projeto_2/microdados_censo_escolar_2023/dados/microdados_ed_basica_2023.csv', header = True, multiLine = True, sep = ';', encoding="latin1")
write_path = '/home/tatiane/Documentos/censo_escolar2/projeto_2/analise2/'

In [3]:
pessoal.completudeSchema(censo)

Qtd. registros: 217625 | Quantidade de colunas:  408
root
 |-- NU_ANO_CENSO: string (nullable = true)
 |-- NO_REGIAO: string (nullable = true)
 |-- CO_REGIAO: string (nullable = true)
 |-- NO_UF: string (nullable = true)
 |-- SG_UF: string (nullable = true)
 |-- CO_UF: string (nullable = true)
 |-- NO_MUNICIPIO: string (nullable = true)
 |-- CO_MUNICIPIO: string (nullable = true)
 |-- NO_REGIAO_GEOG_INTERM: string (nullable = true)
 |-- CO_REGIAO_GEOG_INTERM: string (nullable = true)
 |-- NO_REGIAO_GEOG_IMED: string (nullable = true)
 |-- CO_REGIAO_GEOG_IMED: string (nullable = true)
 |-- NO_MESORREGIAO: string (nullable = true)
 |-- CO_MESORREGIAO: string (nullable = true)
 |-- NO_MICRORREGIAO: string (nullable = true)
 |-- CO_MICRORREGIAO: string (nullable = true)
 |-- CO_DISTRITO: string (nullable = true)
 |-- NO_ENTIDADE: string (nullable = true)
 |-- CO_ENTIDADE: string (nullable = true)
 |-- TP_DEPENDENCIA: string (nullable = true)
 |-- TP_CATEGORIA_ESCOLA_PRIVADA: string (nullab

#### Criação de um identificador unico

In [3]:
censo = createId(censo, 'ID_UNICO')
censo.select(F.min(F.col('ID_UNICO')), F.max(F.col('ID_UNICO'))).show()

+-------------+-------------+
|min(ID_UNICO)|max(ID_UNICO)|
+-------------+-------------+
|            1|       217625|
+-------------+-------------+



#### Escolha de variáveis

In [4]:
analise2 = censo.select('ID_UNICO', 'NO_REGIAO', 'SG_UF', 'NO_MUNICIPIO', 'NO_ENTIDADE', 'TP_DEPENDENCIA', 'TP_LOCALIZACAO', 'TP_LOCALIZACAO_DIFERENCIADA', 'TP_SITUACAO_FUNCIONAMENTO', 'IN_INTERNET', 'IN_INTERNET_ALUNOS', 'IN_ALIMENTACAO', 'IN_EDUCACAO_INDIGENA', 'TP_INDIGENA_LINGUA')
analise2 = analise2.filter(F.col('SG_UF') == 'BA')
completude(analise2)

for col in analise2.columns:
    print(col)
    analise2.groupBy(col).count().orderBy(F.col(col).asc()).show()
printTimer()

Qtd. registros: 19611 | Quantidade de colunas:  14
ID_UNICO


+--------+-----+
|ID_UNICO|count|
+--------+-----+
|   82236|    1|
|   82237|    1|
|   82238|    1|
|   82239|    1|
|   82240|    1|
|   82241|    1|
|   82242|    1|
|   82243|    1|
|   82244|    1|
|   82245|    1|
|   82246|    1|
|   82247|    1|
|   82248|    1|
|   82249|    1|
|   82250|    1|
|   82251|    1|
|   82252|    1|
|   82253|    1|
|   82254|    1|
|   82255|    1|
+--------+-----+
only showing top 20 rows

NO_REGIAO


+---------+-----+
|NO_REGIAO|count|
+---------+-----+
| Nordeste|19611|
+---------+-----+

SG_UF


+-----+-----+
|SG_UF|count|
+-----+-----+
|   BA|19611|
+-----+-----+

NO_MUNICIPIO


+-----------------+-----+
|     NO_MUNICIPIO|count|
+-----------------+-----+
|            Abaré|   45|
|           Abaíra|    8|
|        Acajutiba|   32|
|         Adustina|   11|
|          Aiquara|    8|
|       Alagoinhas|  159|
|         Alcobaça|   44|
|         Almadina|   24|
|         Amargosa|   51|
| Amélia Rodrigues|   31|
|  América Dourada|   32|
|            Anagé|   50|
|          Andaraí|   29|
|        Andorinha|   34|
|          Angical|   35|
|          Anguera|   34|
|            Antas|   23|
|  Antônio Cardoso|   39|
|Antônio Gonçalves|   28|
|            Aporá|   29|
+-----------------+-----+
only showing top 20 rows

NO_ENTIDADE


...

+-----------+-----+
|IN_INTERNET|count|
+-----------+-----+
|       null| 3574|
|          0| 1601|
|          1|14436|
+-----------+-----+

IN_INTERNET_ALUNOS


+------------------+-----+
|IN_INTERNET_ALUNOS|count|
+------------------+-----+
|              null| 3574|
|                 0|10867|
|                 1| 5170|
+------------------+-----+

IN_ALIMENTACAO


+--------------+-----+
|IN_ALIMENTACAO|count|
+--------------+-----+
|          null| 3574|
|             0| 2333|
|             1|13704|
+--------------+-----+

IN_EDUCACAO_INDIGENA


+--------------------+-----+
|IN_EDUCACAO_INDIGENA|count|
+--------------------+-----+
|                null| 3574|
|                   0|15960|
|                   1|   77|
+--------------------+-----+

TP_INDIGENA_LINGUA


+------------------+-----+
|TP_INDIGENA_LINGUA|count|
+------------------+-----+
|              null|19534|
|                 2|   38|
|                 3|   39|
+------------------+-----+

Tempo de execucao: 0:01:15.051769


#### Tratamento das variáveis

- Remoção de numerações no nome da escola
- Transformando numerações em categorias

In [5]:
print('Quantidade de registros que iniciam com numerações que não fazem parte do nome: ', analise2.filter(F.col("NO_ENTIDADE").rlike("^(\d+\.?){3,}")).count())

analise2.filter(F.col("NO_ENTIDADE").rlike("^(\d+\.?){3,}")).select("NO_ENTIDADE").show(truncate=False)

Quantidade de registros que iniciam com numerações que não fazem parte do nome:  1


+-------------------------------------------------------+
|NO_ENTIDADE                                            |
+-------------------------------------------------------+
|29092191 - COLEGIO ESTADUAL DOM PEDRO II TEMPO INTEGRAL|
+-------------------------------------------------------+



In [8]:
# substituição das numerações por vázio
analise2 = analise2.withColumn("NOME_ENTIDADE", F.regexp_replace(F.col("NO_ENTIDADE"), "^(\d+\.?){3,} (- )?", ''))

print('Validando as modificações: ')
analise2.filter(F.col("NO_ENTIDADE").rlike("^(\d+\.?){3,}")).select("NO_ENTIDADE", "NOME_ENTIDADE").show(truncate=False)

#removendo a variável anterior
analise2 = analise2.drop('NO_ENTIDADE')

Validando as modificações: 


+-------------------------------------------------------+--------------------------------------------+
|NO_ENTIDADE                                            |NOME_ENTIDADE                               |
+-------------------------------------------------------+--------------------------------------------+
|29092191 - COLEGIO ESTADUAL DOM PEDRO II TEMPO INTEGRAL|COLEGIO ESTADUAL DOM PEDRO II TEMPO INTEGRAL|
+-------------------------------------------------------+--------------------------------------------+



#### Criação e aplicação de dicionário

In [9]:
from pyspark.sql import functions as F
from itertools import chain

mapas = {
    "TP_DEPENDENCIA" : {1: "Federal", 2: "Estadual", 3: "Municipal", 4: "Privada"},
    "TP_LOCALIZACAO" : {1: "Urbana", 2: "Rural"},
    "TP_LOCALIZACAO_DIFERENCIADA" : {0: "Não se aplica", 1: "Área de assentamento", 2: "Terra índigena", 3: "comunidade remanescente de quilombos", 4: "comunidades tradicionais"},
    "TP_SITUACAO_FUNCIONAMENTO" : {1: "Em atividade", 2: "Paralisada", 3: "Extinta (ano do censo)", 4: "Extinta (anos anteriores)"},
    "IN_INTERNET" : {0: "Não", 1: "Sim"},
    "IN_INTERNET_ALUNOS" : {0: "Não", 1: "Sim"},
    "IN_ALIMENTACAO" : {0: "Não oferece", 1: "Oferece"},
    "IN_EDUCACAO_INDIGENA" : {0: "Não", 1: "Sim"},
    "TP_INDIGENA_LINGUA" : {1: "Somente em Língua Indígena", 2: "Somente em Língua Portuguesa", 3: "Língua Indígena e Portuguesa", 4: "Não aplicável"},
}

In [10]:
analise_final = analise2
for col, dic in mapas.items():
    mapping_expr = F.create_map([F.lit(x) for x in chain(*dic.items())])
    analise_final = analise_final.withColumn(col, mapping_expr[F.col(col)])

In [11]:
for col in analise_final.columns:
    print(col)
    analise_final.groupBy(col).count().orderBy(F.col(col).asc()).show()

ID_UNICO


+--------+-----+
|ID_UNICO|count|
+--------+-----+
|   82236|    1|
|   82237|    1|
|   82238|    1|
|   82239|    1|
|   82240|    1|
|   82241|    1|
|   82242|    1|
|   82243|    1|
|   82244|    1|
|   82245|    1|
|   82246|    1|
|   82247|    1|
|   82248|    1|
|   82249|    1|
|   82250|    1|
|   82251|    1|
|   82252|    1|
|   82253|    1|
|   82254|    1|
|   82255|    1|
+--------+-----+
only showing top 20 rows

NO_REGIAO


+---------+-----+
|NO_REGIAO|count|
+---------+-----+
| Nordeste|19611|
+---------+-----+

SG_UF


+-----+-----+
|SG_UF|count|
+-----+-----+
|   BA|19611|
+-----+-----+

NO_MUNICIPIO


+-----------------+-----+
|     NO_MUNICIPIO|count|
+-----------------+-----+
|            Abaré|   45|
|           Abaíra|    8|
|        Acajutiba|   32|
|         Adustina|   11|
|          Aiquara|    8|
|       Alagoinhas|  159|
|         Alcobaça|   44|
|         Almadina|   24|
|         Amargosa|   51|
| Amélia Rodrigues|   31|
|  América Dourada|   32|
|            Anagé|   50|
|          Andaraí|   29|
|        Andorinha|   34|
|          Angical|   35|
|          Anguera|   34|
|            Antas|   23|
|  Antônio Cardoso|   39|
|Antônio Gonçalves|   28|
|            Aporá|   29|
+-----------------+-----+
only showing top 20 rows

TP_DEPENDENCIA


...

+------------------+-----+
|IN_INTERNET_ALUNOS|count|
+------------------+-----+
|              null| 3574|
|               Não|10867|
|               Sim| 5170|
+------------------+-----+

IN_ALIMENTACAO


+--------------+-----+
|IN_ALIMENTACAO|count|
+--------------+-----+
|          null| 3574|
|   Não oferece| 2333|
|       Oferece|13704|
+--------------+-----+

IN_EDUCACAO_INDIGENA


+--------------------+-----+
|IN_EDUCACAO_INDIGENA|count|
+--------------------+-----+
|                null| 3574|
|                 Não|15960|
|                 Sim|   77|
+--------------------+-----+

TP_INDIGENA_LINGUA


+--------------------+-----+
|  TP_INDIGENA_LINGUA|count|
+--------------------+-----+
|                null|19534|
|Língua Indígena e...|   39|
|Somente em Língua...|   38|
+--------------------+-----+

NOME_ENTIDADE


+--------------------+-----+
|       NOME_ENTIDADE|count|
+--------------------+-----+
|1 G S FRANCISCO D...|    1|
|A CASA AMARELA ES...|    1|
|A F - CRECHE E ES...|    1|
|     AABB COMUNIDADE|    2|
|AAC DE PRAIA DO F...|    1|
|ABA - AMANDO BRIN...|    1|
|ABC SEMEANDO A ES...|    1|
|      ABDON CALDEIRA|    1|
|ABRACE - ASSOCIAC...|    1|
|AC CENTRO EDUCACI...|    1|
|ACBEU MAPLE BEAR ...|    1|
|ADELIA MARIA SALV...|    1|
|ADELINA BORGES DE...|    1|
|ADELINO DE SOUZA ...|    1|
|ADILELMO MARCELIN...|    1|
|      AGENOR BRITO I|    1|
|          AGROVILA I|    1|
|AHPIC CRECHE ESPE...|    1|
|ALBERTO MATEUS AM...|    1|
|  ALICE ALVES BORGES|    1|
+--------------------+-----+
only showing top 20 rows



In [12]:
analise_final = analise_final.withColumnRenamed('NOME_ENTIDADE', 'NO_ENTIDADE')
pessoal.completudeSchema(analise_final)

Qtd. registros: 19611 | Quantidade de colunas:  14
root
 |-- ID_UNICO: long (nullable = false)
 |-- NO_REGIAO: string (nullable = true)
 |-- SG_UF: string (nullable = true)
 |-- NO_MUNICIPIO: string (nullable = true)
 |-- TP_DEPENDENCIA: string (nullable = true)
 |-- TP_LOCALIZACAO: string (nullable = true)
 |-- TP_LOCALIZACAO_DIFERENCIADA: string (nullable = true)
 |-- TP_SITUACAO_FUNCIONAMENTO: string (nullable = true)
 |-- IN_INTERNET: string (nullable = true)
 |-- IN_INTERNET_ALUNOS: string (nullable = true)
 |-- IN_ALIMENTACAO: string (nullable = true)
 |-- IN_EDUCACAO_INDIGENA: string (nullable = true)
 |-- TP_INDIGENA_LINGUA: string (nullable = true)
 |-- NO_ENTIDADE: string (nullable = true)



#### Escrita

In [13]:
writeSingleCsv(analise2, write_path)

('/home/tatiane/Documentos/censo_escolar2/projeto_2/analise2/analise2.csv',
 3.1110839520006266,
 3.3340473099997325)

### Finalização do notebook

In [14]:
executionTime()

Tempo de execucao ate este ponto: 0:07:41.386482


In [16]:
sc.stop()